In [1]:
import torch
torch.cuda.synchronize()
torch.cuda.empty_cache()

https://huggingface.co/docs/transformers/main/model_doc/roberta#transformers.RobertaForMultipleChoice

In [2]:
from transformers import AutoTokenizer, RobertaForMultipleChoice
import torch

tokenizer = AutoTokenizer.from_pretrained("FacebookAI/roberta-base")
model = RobertaForMultipleChoice.from_pretrained("FacebookAI/roberta-base")

prompt = "In Italy, pizza served in formal settings, such as at a restaurant, is presented unsliced."
choice0 = "It is eaten with a fork and a knife."
choice1 = "It is eaten while held in the hand."
labels = torch.tensor(0).unsqueeze(0)  # choice0 is correct (according to Wikipedia ;)), batch size 1

encoding = tokenizer([prompt, prompt], [choice0, choice1], return_tensors="pt", padding=True)
outputs = model(**{k: v.unsqueeze(0) for k, v in encoding.items()}, labels=labels)  # batch size is 1

for k, t in encoding.items():
    print(k, t.shape)

for k, t in outputs.items():
    print(k, t.shape)


/home/aaslab4080/anaconda3/envs/xinbao_llm_final/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Some weights of RobertaForMultipleChoice were not initialized from the model checkpoint at FacebookAI/roberta-base and are newly initialized: ['classifier.bias', 'classifier.weight', 'roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


input_ids torch.Size([2, 35])
attention_mask torch.Size([2, 35])
loss torch.Size([])
logits torch.Size([1, 2])


In [3]:
model.to(device = 'cuda')

RobertaForMultipleChoice(
  (roberta): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(50265, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0-11): 12 x RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (La

## Data Prepare

In [4]:
# 1. Read and Parse Data
import torch
from datasets import load_dataset
from torch.utils.data import Dataset, DataLoader

dataset = load_dataset("ehovy/race", "middle")

print(dataset)

class CustomDataset(Dataset):
    def __init__(self, data):
        self.data = data
    
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, i):
        example = self.data[i]
        prompt = f'{example['article']}'
        options = [f'{example['question']}\n\n{option}' for option in example['options']]
        label = ord(example['answer']) - ord('A')
        encoding = tokenizer([prompt] * len(options), options, return_tensors="pt", padding=True, truncation = 'only_first', max_length=512)
        encoding['labels'] = label
        return encoding

dataset = {
    'test': CustomDataset(dataset['test']),
    'train': CustomDataset(dataset['train']),
    'validation': CustomDataset(dataset['validation'])
}


DatasetDict({
    test: Dataset({
        features: ['example_id', 'article', 'answer', 'question', 'options'],
        num_rows: 1436
    })
    train: Dataset({
        features: ['example_id', 'article', 'answer', 'question', 'options'],
        num_rows: 25421
    })
    validation: Dataset({
        features: ['example_id', 'article', 'answer', 'question', 'options'],
        num_rows: 1436
    })
})


In [5]:
# 5. Create DataLoader
from torch.nn.utils.rnn import pad_sequence
from collections import defaultdict

def _pad_sequence(tensors: list[torch.Tensor]):
    tensors = [
        t.transpose(0, 1)
        for t in tensors
    ]
    batch = pad_sequence(tensors, batch_first=True, padding_value=0)
    return batch.transpose(1, 2).contiguous()

def collect_fn(batch):
    ret = defaultdict(list)
    labels = []
    for encoding in batch:
        for k in encoding.keys():
            if k == 'labels':
                labels.append(encoding[k])
            else:
                ret[k].append(encoding[k])
        

    ret = {k: _pad_sequence(tensors) for k, tensors in ret.items()}
    ret['labels'] = torch.tensor(labels, dtype=torch.uint8)
    
    return ret
    
test_dataloader = DataLoader(dataset['test'], batch_size=2, collate_fn=collect_fn, shuffle=True)

for example in test_dataloader:
    for k, tensor in example.items():
        print(k, tensor.shape)
    break

input_ids torch.Size([2, 4, 331])
attention_mask torch.Size([2, 4, 331])
labels torch.Size([2])


## Test

In [6]:
def test(model):
    acc_count = 0
    total_count = 0
    with torch.no_grad():
        model.eval()
        for batch in test_dataloader:
            batch = {k:t.to(device='cuda')for k,t in batch.items()}
            outputs = model(**batch)
            _, preds = outputs.logits.max(dim = 1)
            labels = batch['labels']
            acc_count += (labels == preds).sum().item()
            total_count += labels.shape[0]

    print(acc_count / total_count)

test(model)

0.24233983286908078


## Train

In [7]:
from transformers import TrainingArguments, Trainer

# 訓練參數
training_args = TrainingArguments(
    output_dir="./results",
    logging_steps=1000,
    eval_steps=2000,
    eval_strategy='steps',
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    num_train_epochs=1,
    learning_rate=2e-5,
    weight_decay=0.01,
    logging_dir="./logs",
)

trainer = Trainer(
    data_collator=collect_fn,
    model=model,
    args=training_args,
    train_dataset=dataset['train'],
    eval_dataset=dataset['validation']
)

# 開始訓練
trainer.train()

Step,Training Loss,Validation Loss
2000,1.405700,1.386256
4000,1.399400,1.374313
6000,1.269700,1.374864
8000,1.240800,1.177613
10000,1.189500,1.126991
12000,1.141400,1.195610


TrainOutput(global_step=12711, training_loss=1.2648651648839464, metrics={'train_runtime': 1213.9495, 'train_samples_per_second': 20.941, 'train_steps_per_second': 10.471, 'total_flos': 1.6502812385287416e+16, 'train_loss': 1.2648651648839464, 'epoch': 1.0})

In [8]:
test(model)

0.541782729805014


5340MiB